In [19]:
import pandas as pd
import re

In [20]:
nocnok = pd.read_csv("/home/mario/Proyectos_Mario/proyecto_alonso/datos_extraidos/casas_nocnok.csv")

In [21]:
# Se eliminan todas las casas que tienen un remate bancario o hipotecario
# ya que no representan el valor real 
casas_remate = list(nocnok[nocnok.description.str.contains("remate|bancaria|recuperación|hipotecaria", case=False)].index)
nocnok = nocnok.drop(index=casas_remate)
nocnok = nocnok.reset_index(drop="first")

In [22]:
# Todos los valores de la columna parking son nulos
# De momento se eliminará la columna para continuar con 
# la limpieza de los datos
# La columna address también tiene valores nulos
nocnok.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          103 non-null    object 
 1   seller         103 non-null    object 
 2   property_type  103 non-null    object 
 3   address        102 non-null    object 
 4   price          103 non-null    int64  
 5   bedrooms       103 non-null    int64  
 6   bathrooms      103 non-null    int64  
 7   built_Area     103 non-null    int64  
 8   land_Area      103 non-null    object 
 9   parking        0 non-null      float64
 10  description    103 non-null    object 
dtypes: float64(1), int64(4), object(6)
memory usage: 9.0+ KB


In [23]:
# Se elimina la columna parking
nocnok = nocnok.drop(columns="parking")
# Elimino los valores nulos de address
nocnok = nocnok.dropna(ignore_index=True)

In [24]:
# Adecuo los valores de la columna adddress para hacer una búsqueda efectiva de los zip codes
nocnok["address"] = nocnok["address"].apply(lambda x: x.lower()\
                                                        .replace("á", "a")\
                                                        .replace("é", "e")\
                                                        .replace("í", "i")\
                                                        .replace("ó", "o")\
                                                        .replace("ú", "u"))
nocnok["address"] = nocnok["address"].str.replace(", oaxaca de juarez, oaxaca", "").str.strip()


### Creando la columna zip_code

In [25]:
len(nocnok[nocnok.address.str.contains(", oaxaca de juarez, oaxaca", case=False)])

0

In [26]:
# Leo la tabla de códigos postales
tabla_CP = pd.read_html("https://codigo-postal.co/mexico/oaxaca/oaxaca-de-juarez/")
tabla_CP = tabla_CP[0]

# Seleccionar las columas que me sirven 
tabla_CP = tabla_CP.iloc[:, :2]
tabla_CP = tabla_CP.drop_duplicates(ignore_index=True)

# Guardar en listas las colonias y sus C.P.
cp = list(tabla_CP.iloc[:, 0])
direccion = list(tabla_CP.iloc[:, 1])

# Crear un diccionario y Eliminar acentos de los nombres de las direcciones
cp_dict = {}
for i, j in zip(direccion, cp):
    cp_dict[i.lower()\
        .replace("á", "a")\
        .replace("é", "e")\
        .replace("í", "i")\
        .replace("ó", "o")\
        .replace("ú", "u")] = j

In [27]:
# Creo una lista con los datos de zip code para cada propiedad
cp_column = []
for i, d in zip(nocnok.index, nocnok.address):
    regex = re.compile(fr"(.{{0,20}}{d}.{{0,20}})", re.IGNORECASE)
    location_name = list(filter(lambda x: regex.findall(x), cp_dict.keys()))

    try:
        cp_column.append(cp_dict[location_name[0]])
    except:
        cp_column.append(None)

In [28]:
# Creo la columna zip_code con los valores obtenidos de la iteración
nocnok["zip_code"] = cp_column

In [29]:
nocnok.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          102 non-null    object 
 1   seller         102 non-null    object 
 2   property_type  102 non-null    object 
 3   address        102 non-null    object 
 4   price          102 non-null    int64  
 5   bedrooms       102 non-null    int64  
 6   bathrooms      102 non-null    int64  
 7   built_Area     102 non-null    int64  
 8   land_Area      102 non-null    object 
 9   description    102 non-null    object 
 10  zip_code       99 non-null     float64
dtypes: float64(1), int64(4), object(6)
memory usage: 8.9+ KB


In [30]:
# Se eliminan los valores nulos
nocnok = nocnok.drop_duplicates()
nocnok = nocnok.dropna(ignore_index=True)

In [31]:
nocnok.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          83 non-null     object 
 1   seller         83 non-null     object 
 2   property_type  83 non-null     object 
 3   address        83 non-null     object 
 4   price          83 non-null     int64  
 5   bedrooms       83 non-null     int64  
 6   bathrooms      83 non-null     int64  
 7   built_Area     83 non-null     int64  
 8   land_Area      83 non-null     object 
 9   description    83 non-null     object 
 10  zip_code       83 non-null     float64
dtypes: float64(1), int64(4), object(6)
memory usage: 7.3+ KB


In [32]:
len(nocnok.drop_duplicates())

83

### Creando la columna parking

In [33]:
# Con esta función localizo el texto donde especifica los lugares de estacionamiento en la descripción
def econtrar_textos(texto):
    regex = r"(?i)(.{0,10}estacionamientos?.{0,30}|.{0,10}cocheras?.{0,30}|.{0,10}garages?.{0,30}|.{0,10}parkings?.{0,30})"
    expresion = re.findall(regex, texto)
    expresion = "".join(expresion)
    if len(expresion) > 0:
        valor_de_celda = expresion
    else:
        valor_de_celda = "cero"
    return valor_de_celda

In [34]:
# Función para encontrar en la descripción el número de lugares de estacionamiento
def asignar_valor_estacionamiento(texto):
    # Buscar dígitos numéricos en el texto
    if texto != "cero":    
        numeros_encontrados = re.findall(r'\d+', texto)

        if numeros_encontrados:
            # Tomar el primer número encontrado y convertirlo a entero
            return int(numeros_encontrados[0])
        elif "para dos" in texto.lower():
            return 2
        elif "para tres" in texto.lower():
            return 3
        elif "para cuatro" in texto.lower():
            return 4
        elif "para cinco" in texto.lower():
            return 5
        else:
            return 1

In [35]:
# Aplicar las funciones para buscar los valores de parking
nocnok["parking"] = nocnok["description"].apply(econtrar_textos)
nocnok["parking"] = nocnok["parking"].apply(asignar_valor_estacionamiento)

In [36]:
nocnok.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          83 non-null     object 
 1   seller         83 non-null     object 
 2   property_type  83 non-null     object 
 3   address        83 non-null     object 
 4   price          83 non-null     int64  
 5   bedrooms       83 non-null     int64  
 6   bathrooms      83 non-null     int64  
 7   built_Area     83 non-null     int64  
 8   land_Area      83 non-null     object 
 9   description    83 non-null     object 
 10  zip_code       83 non-null     float64
 11  parking        55 non-null     float64
dtypes: float64(2), int64(4), object(6)
memory usage: 7.9+ KB


In [37]:
nocnok.to_csv("datos_nocnok.csv", index=False)